# ETL (Extrair, Transformar e Carregar) da camada Silver para Gold.

Este notebook realiza o processo de ETL para transformar e carregar os dados da camada Silver para a camada Gold no data lake. A camada Gold é otimizada para consultas analíticas e relatórios, garantindo que os dados estejam prontos para uso por ferramentas de BI e análise avançada.


### Configuração do Ambiente de Desenvolvimento

In [ ]:
!pip install pyspark
!pip install psycopg2
!pip install pandas

# Criação e Carga das Dimensões (Star Schema)

Esta seção tem como objetivo realizar a **criação e carga das tabelas dimensão**
da camada Gold, seguindo o modelo **Star Schema**, a partir de dados previamente
tratados na camada Silver.


### Importação de Bibliotecas

Nesta seção são importadas as bibliotecas necessárias para:
- manipulação de dados com PySpark
- criação de colunas derivadas
- conexão com o banco PostgreSQL

In [ ]:
import pandas as pd
import psycopg2

from psycopg2.extras import execute_batch
from psycopg2 import extras

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, concat_ws, year, month, dayofmonth, quarter, 
    dayofweek, when, date_format, monotonically_increasing_id,
    lit, trim, upper
)

from pyspark.sql import functions as F

from pyspark.sql.types import StringType, IntegerType, DateType, DecimalType

### Configuração do Ambiente

Nesta etapa são configurados:
- a sessão Spark
- a conexão JDBC com o PostgreSQL
- o schema de destino da camada Gold

In [ ]:
spark = SparkSession.builder \
    .appName("SilverToGold") \
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")

POSTGRES_HOST = "localhost"
POSTGRES_PORT = "5432"
POSTGRES_DB = "cat_db"
POSTGRES_USER = "admin"
POSTGRES_PASSWORD = "admin"

conn_params = {
    "host": POSTGRES_HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "port": POSTGRES_PORT
}

connection_properties = {
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "driver": "org.postgresql.Driver"
}

try:
    conn = psycopg2.connect(**conn_params)
    
    query = "SELECT * FROM ACIDENTE"
    
    pdf = pd.read_sql(query, conn)
    df_silver = spark.createDataFrame(pdf)
    
    print("Dados da camada Silver carregados")
    df_silver.show()

except Exception as e:
    print(f"Erro na leitura: {e}")
finally:
    if conn:
        conn.close()
        
GOLD_SCHEMA = "gold"

### Função Genérica de Carga de Dimensões

Esta função padroniza o processo de carga das dimensões, garantindo:
- remoção de duplicidades
- preservação da business key
- inserção no schema Gold
- validação básica pós-carga

In [ ]:
def load_dim_with_psycopg2(table_name, pg_conn_params, spark):
    query = f"SELECT * FROM {table_name}"

    with psycopg2.connect(**pg_conn_params) as conn:
        pdf = pd.read_sql(query, conn)

    df_dim_loaded = spark.createDataFrame(pdf)
    return df_dim_loaded

In [ ]:
def save_dimension(
    df_dim,
    dim_name,
    id_col_silver,
    other_cols,
    pg_conn_params,
    cols_to_drop=None
):
    """
    Salva dados em uma dimensão no PostgreSQL usando psycopg2,
    preservando a business key.
    Retorna um DataFrame Spark da dimensão carregada (com surrogate keys).
    """

    table_name = f"{GOLD_SCHEMA}.dim_{dim_name}"
    business_key_col = f"chv_{dim_name}_org"

    # Preparação da dimensão no Spark
    df_dim_unique = (
        df_dim
        .select(col(id_col_silver).alias(business_key_col), *other_cols)
        .distinct()
        .dropna(subset=[business_key_col])
    )

    if cols_to_drop:
        df_dim_unique = df_dim_unique.drop(*cols_to_drop)

    print(f"\n---> Criando e carregando Dimensão: {table_name}")
    count_unique = df_dim_unique.count()
    print(f"     Registros únicos: {count_unique}")

    if count_unique == 0:
        print("DataFrame vazio")
        return None

    # Converter para Pandas para inserção via psycopg2
    pdf = df_dim_unique.toPandas()

    columns = list(pdf.columns)
    cols_sql = ", ".join(columns)
    values_sql = ", ".join([f"%({c})s" for c in columns])

    insert_sql = f"""
        INSERT INTO {table_name} ({cols_sql})
        VALUES ({values_sql})
        ON CONFLICT ({business_key_col}) DO NOTHING
    """

    try:
        # Conexão PostgreSQL
        with psycopg2.connect(**pg_conn_params) as conn:
            with conn.cursor() as cur:
                execute_batch(
                    cur,
                    insert_sql,
                    pdf.to_dict(orient="records"),
                    page_size=1000
                )
            conn.commit()

        print("Inserção concluída")

        # Recarregar dimensão com surrogate keys via Spark
        df_dim_loaded = load_dim_with_psycopg2(
            table_name=table_name,
            pg_conn_params=pg_conn_params,
            spark=spark
        )


        count_check = df_dim_loaded.count()

        if count_check >= count_unique:
            print(f"Dimensão carregada. Registros confirmados: {count_check}")
        else:
            print(f"Confirmados ({count_check}) < esperados ({count_unique})")

        return df_dim_loaded

    except Exception as e:
        print(f"Erro ao gravar dimensão {table_name}: {e}")
        raise

### Dimensão Tempo

A dimensão tempo é utilizada para:
- data do acidente
- data de emissão da CAT
- data de nascimento do trabalhador

Ela permite análises temporais como sazonalidade, tendências e comparações anuais.

In [ ]:
def create_dim_tempo(df_silver):    
    df_datas = df_silver.select("data_acidente_referencia").distinct() \
        .union(df_silver.select("data_emissao_cat").distinct()) \
        .union(df_silver.select("data_nascimento").distinct()) \
        .withColumnRenamed("data_acidente_referencia", "data") \
        .distinct() \
        .dropna()
    
    df_tempo = df_datas.select(
        col("data"),
        dayofmonth("data").alias("dia"),
        month("data").alias("mes"),
        date_format("data", "MMMM").alias("nome_mes"),
        quarter("data").alias("trimestre"),
        year("data").alias("ano"),
        dayofweek("data").alias("dia_semana"),
        when(dayofweek("data").isin(1, 7), True).otherwise(False).alias("is_fim_semana")
    )
    
    return save_dimension(
        df_tempo,
        "tempo",
        "data",
        ["dia", "mes", "nome_mes", "trimestre", "ano", "dia_semana", "is_fim_semana"],
        conn_params
    )

### Dimensão Trabalhador

Representa características demográficas e ocupacionais do trabalhador
no momento do acidente.

In [ ]:
def create_dim_trabalhador(df_silver):
    df_trabalhador = df_silver.select(
        concat_ws("_", 
                  upper(trim(col("sexo"))),
                  col("cbo_codigo_descricao"),
                  col("data_nascimento")
        ).alias("srk_trab"),
        upper(trim(col("sexo"))).alias("sexo"),
        col("cbo_codigo_descricao").alias("srk_cbo"),
        col("data_nascimento").alias("srk_temp_nasc")
    )
    
    return save_dimension(
        df_trabalhador,
        "trabalhador",
        "srk_trab",
        ["sexo", "srk_cbo", "srk_temp_nasc"],
        conn_params
    )

### Dimensão Empregador

A dimensão empregador representa as características da empresa
responsável pelo vínculo de trabalho no momento do acidente.

Esta dimensão referencia:
- a dimensão CNAE (atividade econômica)
- a dimensão Município (localização do empregador)

Sua criação ocorre após a carga das dimensões independentes,
garantindo integridade referencial.

In [ ]:
def create_dim_empregador(df_silver):
    df_empregador = df_silver.select(
        concat_ws("_", 
                  col("cnae_empregador"),
                  col("municipio_empregador")
        ).alias("srk_empreg"),
        col("cnae_empregador").alias("srk_cnae"),
        col("municipio_empregador").alias("srk_munic")
    )
    
    return save_dimension(
        df_empregador,
        "empregador",
        "srk_empreg",
        ["srk_cnae", "srk_munic"],
        conn_params
    )

### Dimensões de Classificação

As dimensões a seguir representam classificações oficiais utilizadas
para padronização e análise estatística.

In [ ]:
def create_dim_cbo(df_silver):
    df_cbo = df_silver.select(
        col("cbo_codigo_descricao").alias("srk_cbo"),
        col("cbo_codigo_descricao").alias("codigo"),
        col("cbo_codigo_descricao").alias("descricao")
    )
    
    return save_dimension(
        df_cbo,
        "cbo",
        "srk_cbo",
        ["codigo", "descricao"],
        conn_params
    )


def create_dim_cnae(df_silver):
    df_cnae = df_silver.select(
        col("cnae_empregador").alias("srk_cnae"),
        col("cnae_empregador").alias("codigo"),
        col("cnae_empregador_descricao").alias("descricao")
    )
    
    return save_dimension(
        df_cnae,
        "cnae",
        "srk_cnae",
        ["codigo", "descricao"],
        conn_params
    )


def create_dim_cid10(df_silver):
    df_cid = df_silver.select(
        col("cid_10").alias("srk_cid10"),
        col("cid_10").alias("codigo"),
        col("cid_10").alias("descricao")
    )
    
    return save_dimension(
        df_cid,
        "cid10",
        "srk_cid10",
        ["codigo", "descricao"],
        conn_params
    )

### Dimensão Município

A dimensão município é utilizada para:
- local do acidente
- local do empregador

In [ ]:
def create_dim_municipio(df_silver):
    df_mun_acidente = df_silver.select(
        col("uf_municipio_acidente").alias("codigo_ibge"),
        col("uf_municipio_acidente").alias("nome"),
        col("uf_municipio_acidente").alias("uf")
    )
    
    df_mun_empregador = df_silver.select(
        col("municipio_empregador").alias("codigo_ibge"),
        col("municipio_empregador").alias("nome"),
        col("uf_municipio_empregador").alias("uf")
    )
    
    df_municipio = df_mun_acidente.union(df_mun_empregador) \
        .distinct() \
        .dropna(subset=["codigo_ibge"])
    
    return save_dimension(
        df_municipio,
        "municipio",
        "codigo_ibge",
        ["nome", "uf"],
        conn_params
    )

### Dimensões de Caracterização do Acidente

Estas dimensões descrevem o contexto e as causas do acidente de trabalho.


In [ ]:
def create_dim_tipo_acidente(df_silver):    
    df_tipo_acidente = df_silver.select(
        col("tipo_acidente").alias("srk_tp_acdt"),
        col("tipo_acidente").alias("descricao")
    )
    
    return save_dimension(
        df_tipo_acidente,
        "tipo_acidente",
        "srk_tp_acdt",
        ["descricao"],
        conn_params
    )


def create_dim_lesao(df_silver):
    df_lesao = df_silver.select(
        concat_ws("_", 
                  col("natureza_lesao"), 
                  col("parte_corpo_atingida")
        ).alias("srk_lesao"),
        col("natureza_lesao").alias("natureza_lesao"),
        col("parte_corpo_atingida").alias("parte_corpo_atingida")
    )
    
    return save_dimension(
        df_lesao,
        "lesao",
        "srk_lesao",
        ["natureza_lesao", "parte_corpo_atingida"],
        conn_params
    )


def create_dim_agente_causador(df_silver):
    df_agente = df_silver.select(
        col("agente_causador_acidente").alias("srk_agnt_csdr"),
        col("agente_causador_acidente").alias("descricao")
    )
    
    return save_dimension(
        df_agente,
        "agente_causador",
        "srk_agnt_csdr",
        ["descricao"],
        conn_params
    )

## Execução da Carga das Dimensões

Nesta etapa é executado o processo completo de criação e carga das dimensões,
respeitando a ordem de dependência entre elas.

In [ ]:
try:
    dim_tempo = create_dim_tempo(df_silver)          
    dim_cbo = create_dim_cbo(df_silver)            
    dim_municipio = create_dim_municipio(df_silver)      
    dim_cnae = create_dim_cnae(df_silver)           
    dim_tipo_acidente = create_dim_tipo_acidente(df_silver)  
    dim_lesao = create_dim_lesao(df_silver)          
    dim_agente_causador = create_dim_agente_causador(df_silver)
    dim_cid10 = create_dim_cid10(df_silver)          

    dim_trabalhador = create_dim_trabalhador(df_silver)
    dim_empregador = create_dim_empregador(df_silver)
        
    print("Dimensões foram carregadas")
        
except Exception as e:
    print(f"\nERRO NA CARGA DAS DIMENSÕES: {e}")
    raise e

### Preparando dados da camada Silver

In [ ]:
df_base = df_silver.select(
    concat_ws("_",
        col("data_acidente_referencia"),
        col("sexo"),
        col("cbo_codigo_descricao"),
        col("cnae_empregador"),
        col("municipio_empregador")
    ).alias("srk_cat"),
    
    col("data_acidente_referencia").alias("data_acidente"),
    col("data_emissao_cat").alias("data_emissao"),
    col("data_nascimento"),
    
    concat_ws("_", 
        upper(trim(col("sexo"))),
        col("cbo_codigo_descricao"),
        col("data_nascimento")
    ).alias("srk_trab"),
    col("cbo_codigo_descricao").alias("codigo_cbo"),
    
    concat_ws("_", 
        col("cnae_empregador"),
        col("municipio_empregador")
    ).alias("srk_empreg"),
    col("cnae_empregador").alias("codigo_cnae"),
    
    col("uf_municipio_acidente").alias("codigo_municipio_acidente"),
    col("municipio_empregador").alias("codigo_municipio_empregador"),
    
    col("tipo_acidente").alias("codigo_tipo_acidente"),
    col("natureza_lesao").alias("codigo_natureza_lesao"),
    col("parte_corpo_atingida").alias("codigo_parte_corpo"),
    col("agente_causador_acidente").alias("codigo_agente_causador"),
    col("cid_10").alias("codigo_cid10"),
    
    (year(col("data_acidente_referencia")) - year(col("data_nascimento"))).cast(IntegerType()).alias("idade_trabalhador"),
)


print(f"Registros preparados: {df_base.count()}")


## Lookup de Surrogate Keys

Realiza joins (lookups) entre df_base e todas as dimensões
para substituir business keys por surrogate keys (FKs).

- Trocar valores de negócio por IDs técnicos (surrogate keys)
- Criar relacionamentos entre fato e dimensões
- Preparar estrutura final da tabela fato

In [ ]:
df_fact = df_base.join(
    dim_tempo.select(
        col("srk_temp").alias("srk_temp_acdt"),
        col("chv_tempo_org").alias("data_join_acidente")
    ),
    df_base.data_acidente == col("data_join_acidente"),
    "left"
).drop("data_join_acidente", "data_acidente")


df_fact = df_fact.join(
    dim_tempo.select(
        col("srk_temp").alias("srk_temp_emss"),
        col("chv_tempo_org").alias("data_join_emissao")
    ),
    df_fact.data_emissao == col("data_join_emissao"),
    "left"
).drop("data_join_emissao", "data_emissao")


df_fact = df_fact.join(
    dim_tempo.select(
        col("srk_temp").alias("srk_temp_nasc"),
        col("chv_tempo_org").alias("data_join_nascimento")
    ),
    df_fact.data_nascimento == col("data_join_nascimento"),
    "left"
).drop("data_join_nascimento", "data_nascimento")


df_fact = df_fact.join(
    dim_trabalhador.select(
        col("srk_trab").alias("srk_trab_fk"),
        col("chv_trabalhador_org").alias("id_trab_join")
    ),
    df_fact.srk_trab == col("id_trab_join"),
    "left"
).drop("id_trab_join", "srk_trab").withColumnRenamed("srk_trab_fk", "srk_trab")


df_fact = df_fact.join(
    dim_cbo.select(
        col("srk_cbo").alias("srk_cbo_fk"),
        col("chv_cbo_org").alias("codigo_cbo_join")
    ),
    df_fact.codigo_cbo == col("codigo_cbo_join"),
    "left"
).drop("codigo_cbo_join", "codigo_cbo").withColumnRenamed("srk_cbo_fk", "srk_cbo")


df_fact = df_fact.join(
    dim_empregador.select(
        col("srk_empreg").alias("srk_empreg_fk"),
        col("chv_empregador_org").alias("id_emp_join")
    ),
    df_fact.srk_empreg == col("id_emp_join"),
    "left"
).drop("id_emp_join", "srk_empreg").withColumnRenamed("srk_empreg_fk", "srk_empreg")


df_fact = df_fact.join(
    dim_cnae.select(
        col("srk_cnae").alias("srk_cnae_fk"),
        col("chv_cnae_org").alias("codigo_cnae_join")
    ),
    df_fact.codigo_cnae == col("codigo_cnae_join"),
    "left"
).drop("codigo_cnae_join", "codigo_cnae").withColumnRenamed("srk_cnae_fk", "srk_cnae")


df_fact = df_fact.join(
    dim_municipio.select(
        col("srk_munic").alias("srk_munic_acdt"),
        col("chv_municipio_org").alias("mun_acidente_join")
    ),
    df_fact.codigo_municipio_acidente == col("mun_acidente_join"),
    "left"
).drop("mun_acidente_join", "codigo_municipio_acidente")


df_fact = df_fact.join(
    dim_municipio.select(
        col("srk_munic").alias("srk_munic_empreg"),
        col("chv_municipio_org").alias("mun_empregador_join")
    ),
    df_fact.codigo_municipio_empregador == col("mun_empregador_join"),
    "left"
).drop("mun_empregador_join", "codigo_municipio_empregador")


df_fact = df_fact.join(
    dim_tipo_acidente.select(
        col("srk_tp_acdt").alias("srk_tp_acdt_fk"),
        col("chv_tipo_acidente_org").alias("tipo_acidente_join")
    ),
    df_fact.codigo_tipo_acidente == col("tipo_acidente_join"),
    "left"
).drop("tipo_acidente_join", "codigo_tipo_acidente").withColumnRenamed("srk_tp_acdt_fk", "srk_tp_acdt")


df_fact = df_fact.withColumn(
    "lesao_join_key",
    concat_ws("_", col("codigo_natureza_lesao"), col("codigo_parte_corpo"))
)

df_fact = df_fact.join(
    dim_lesao.select(
        col("srk_lesao").alias("srk_lesao_fk"),
        col("chv_lesao_org").alias("lesao_join")
    ),
    df_fact.lesao_join_key == col("lesao_join"),
    "left"
).drop("lesao_join", "lesao_join_key", "codigo_natureza_lesao", "codigo_parte_corpo").withColumnRenamed("srk_lesao_fk", "srk_lesao")


df_fact = df_fact.join(
    dim_agente_causador.select(
        col("srk_agnt_csdr").alias("srk_agnt_csdr_fk"),
        col("chv_agente_causador_org").alias("agente_join")
    ),
    df_fact.codigo_agente_causador == col("agente_join"),
    "left"
).drop("agente_join", "codigo_agente_causador").withColumnRenamed("srk_agnt_csdr_fk", "srk_agnt_csdr")


df_fact = df_fact.join(
    dim_cid10.select(
        col("srk_cid10").alias("srk_cid10_fk"),
        col("chv_cid10_org").alias("cid_join")
    ),
    df_fact.codigo_cid10 == col("cid_join"),
    "left"
).drop("cid_join", "codigo_cid10").withColumnRenamed("srk_cid10_fk", "srk_cid10")

## Montagem da Tabela Fato

In [ ]:
df_fact_final = df_fact.select(
    col("srk_cat").alias("chv_cat_org"),
    
    "srk_temp_acdt",
    "srk_temp_emss",
    "srk_temp_nasc",
    
    "srk_trab",
    "srk_cbo",
    "srk_empreg",
    "srk_cnae",
    
    "srk_munic_acdt",
    "srk_munic_empreg",
    "srk_tp_acdt",
    "srk_lesao",
    "srk_agnt_csdr",
    "srk_cid10",
    "idade_trabalhador"\
    
).distinct()
count_fact = df_fact_final.count()
print(f"Registros na tabela fato: {count_fact}")

### Carga da Tabela Fato

In [ ]:
colunas = df_fact_final.columns
fact_table_name = f"{GOLD_SCHEMA}.fato_acidente_trabalho"

print(f"Colunas: {len(colunas)}")
print(f"Colunas a inserir: {colunas}")
print(f"\nPreparando dados para inserção...")

dados_para_inserir = [tuple(row) for row in df_fact_final.collect()]

conn = None
cursor = None

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    query = f"INSERT INTO {fact_table_name} ({', '.join(colunas)}) VALUES %s"
    
    extras.execute_values(
        cursor,
        query,
        dados_para_inserir,
        template=None,
        page_size=1000
    )
    
    conn.commit()
    print("Registros concluída com sucesso!")
except Exception as e:
    print(f"\n❌ Erro ao inserir dados: {e}")
    if conn:
        conn.rollback()
    raise e
    
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    spark.stop()